# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)


00%|███████████████████████████████████████| 5/5 [01:44<00:00, 20.82s/it]

In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: Samsung Vacuum Deals: Up to $200 off + free shipping\nDetails: Save on Samsung's range of stick and robot vacuums. Deals start from $259.99 with the pictured Samsung Jet 60 Pet Cordless Stick Vacuum Cleaner ($70 off). Shop Now at Samsung\nFeatures: \nURL: https://www.dealnews.com/Samsung-Vacuum-Deals-Up-to-200-off-free-shipping/21706685.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Samsung S90D Series QN42S90DAEXZA 42" 4K HDR 144Hz OLED Smart TV for $1,100 + free shipping
Details: That's $300 off, and the best price we could find for this OLED TV. Buy Now at Samsung
Features: 3840x2160 (4K) native resolution OLED HDR 144Hz refresh rate w/ AMD FreeSync Tizen smart OS 4 HDMI inputs Model: QN42S90DAEXZA
URL: https://www.dealnews.com/products/Sams

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description="The Fossil Men's Gen 6 44mm Wellness Smartwatch is designed to help you stay on top of your health and fitness goals. This smartwatch includes features such as an activity tracker for monitoring distance, GPS, step counts, and sleep patterns. Running on Wear OS by Google, it is compatible with Alexa and has an Always-On display for easy access to information. Built to be both stylish and functional, this watch is a perfect companion for daily wear.", price=50.0, url='https://www.dealnews.com/products/Fossil/Fossil-Mens-Gen-6-44-mm-Wellness-Smartwatch/484830.html?iref=rss-c142')

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
agent = ScannerAgent()
result = agent.scan()

In [15]:
result

DealSelection(deals=[Deal(product_description='The Samsung S90D Series QN42S90DAEXZA is a stunning 42-inch OLED Smart TV that offers an immersive viewing experience with its native 4K resolution of 3840x2160. This television features a high dynamic range (HDR) and a 144Hz refresh rate, ensuring smooth motion and vibrant colors. With AMD FreeSync technology, it minimizes lag for a better gaming experience. The smart operating system, Tizen, provides access to various streaming services and apps, making it a versatile entertainment hub.', price=1100.0, url='https://www.dealnews.com/products/Samsung/Samsung-S90-D-Series-QN42-S90-DAEXZA-42-4-K-HDR-144-Hz-OLED-Smart-TV/485596.html?iref=rss-c142'), Deal(product_description='The Lensbaby Velvet 85 is an innovative 85mm f/1.8 lens designed for Micro Four Thirds cameras, which excels in creating beautiful, artistic images with a dreamy, velvety characteristic. It features a unique optical design that provides exceptional sharpness and stunning 